In [1]:
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm
from torchinfo import summary
from sklearn.metrics import accuracy_score

In [2]:
# --- 1. Load Pre-processed DataFrames ---
print("Loading pre-processed DataFrames from disk...")
PROCESSED_DATA_DIR = '../data/processed'
train_df = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'train.parquet'))
val_df = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'val.parquet'))
print("DataFrames loaded successfully.")

Loading pre-processed DataFrames from disk...
DataFrames loaded successfully.


In [3]:
# --- 2. Recreate Data Pipeline (from previous step) ---
print("Re-creating the data pipeline...")
# Transformations
IMAGE_SIZE = 224
BATCH_SIZE = 32
train_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
val_test_transforms = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
# Dataset Class


class PlantVillageDataset(Dataset):
    def __init__(self, dataframe, class_to_idx_map, transform=None):
        self.df = dataframe
        self.transform = transform
        self.class_to_idx_map = class_to_idx_map

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        image_path = self.df.iloc[idx]['filepath']
        label_str = self.df.iloc[idx]['label']
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        label_idx = self.class_to_idx_map[label_str]
        return image, label_idx


# Mappings, Datasets, and DataLoaders
class_to_idx = {label: i for i, label in enumerate(train_df['label'].unique())}
NUM_CLASSES = len(class_to_idx)

train_dataset = PlantVillageDataset(
    train_df, class_to_idx, transform=train_transforms)
val_dataset = PlantVillageDataset(
    val_df, class_to_idx, transform=val_test_transforms)

train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(dataset=val_dataset,
                        batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
print("Data pipeline ready.")

Re-creating the data pipeline...
Data pipeline ready.


In [4]:
# --- 3. Define the Model ---
print("\nDefining the model architecture...")
# Load a pre-trained EfficientNet-B0 model
model = models.efficientnet_b0(weights='IMAGENET1K_V1')

# Freeze all the parameters in the pre-trained model
for param in model.parameters():
    param.requires_grad = False

# Replace the final classifier layer
# EfficientNet's classifier is at 'model.classifier[1]'
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(
    in_features=num_features, out_features=NUM_CLASSES)

# --- Verification ---
# Use torchinfo to display a summary of the model
# This will show which layers are frozen (not trainable) and which are not.
print("Model summary:")
summary(model, input_size=(BATCH_SIZE, 3, IMAGE_SIZE, IMAGE_SIZE))


Defining the model architecture...
Downloading: "https://download.pytorch.org/models/efficientnet_b0_rwightman-7f5810bc.pth" to C:\Users\Punith/.cache\torch\hub\checkpoints\efficientnet_b0_rwightman-7f5810bc.pth


100%|██████████| 20.5M/20.5M [00:09<00:00, 2.17MB/s]


Model summary:


Layer (type:depth-idx)                                  Output Shape              Param #
EfficientNet                                            [32, 38]                  --
├─Sequential: 1-1                                       [32, 1280, 7, 7]          --
│    └─Conv2dNormActivation: 2-1                        [32, 32, 112, 112]        --
│    │    └─Conv2d: 3-1                                 [32, 32, 112, 112]        (864)
│    │    └─BatchNorm2d: 3-2                            [32, 32, 112, 112]        (64)
│    │    └─SiLU: 3-3                                   [32, 32, 112, 112]        --
│    └─Sequential: 2-2                                  [32, 16, 112, 112]        --
│    │    └─MBConv: 3-4                                 [32, 16, 112, 112]        (1,448)
│    └─Sequential: 2-3                                  [32, 24, 56, 56]          --
│    │    └─MBConv: 3-5                                 [32, 24, 56, 56]          (6,004)
│    │    └─MBConv: 3-6                      

In [5]:
# --- 4. Set up Training Components ---
print("\nSetting up training components...")
# Set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Move the model to the selected device
model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer (only for the unfrozen classifier parameters)
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
print("Loss function and optimizer are set up.")


Setting up training components...
Using device: cpu
Loss function and optimizer are set up.


In [ ]:
# --- 5. Run the Training Loop ---
NUM_EPOCHS = 5  # Let's start with 5 epochs
best_val_accuracy = 0.0
model_save_path = '../best_crop_doctor_model.pth'

print(f"\nStarting training for {NUM_EPOCHS} epochs...")

for epoch in range(NUM_EPOCHS):
    # --- Training Phase ---
    model.train()  # Set the model to training mode
    running_loss = 0.0
    train_preds, train_labels = [], []

    train_progress_bar = tqdm(
        train_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Training]")
    for inputs, labels in train_progress_bar:
        # Move data to the device
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item() * inputs.size(0)
        _, preds = torch.max(outputs, 1)
        train_preds.extend(preds.cpu().numpy())
        train_labels.extend(labels.cpu().numpy())

        # Update progress bar
        train_progress_bar.set_postfix(loss=loss.item())

    train_loss = running_loss / len(train_loader.dataset)
    train_accuracy = accuracy_score(train_labels, train_preds)

    # --- Validation Phase ---
    model.eval()  # Set the model to evaluation mode
    val_running_loss = 0.0
    val_preds, val_labels = [], []

    with torch.no_grad():  # No need to calculate gradients during validation
        val_progress_bar = tqdm(
            val_loader, desc=f"Epoch {epoch+1}/{NUM_EPOCHS} [Validating]")
        for inputs, labels in val_progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            val_preds.extend(preds.cpu().numpy())
            val_labels.extend(labels.cpu().numpy())
            val_progress_bar.set_postfix(loss=loss.item())

    val_loss = val_running_loss / len(val_loader.dataset)
    val_accuracy = accuracy_score(val_labels, val_preds)

    print(f"Epoch {epoch+1}/{NUM_EPOCHS} -> "
          f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.4f} | "
          f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    # Save the model if it has the best validation accuracy so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        torch.save(model.state_dict(), model_save_path)
        print(
            f"-> New best model saved to '{model_save_path}' with accuracy: {best_val_accuracy:.4f}")

print("\nTraining complete!")
print(f"Best validation accuracy: {best_val_accuracy:.4f}")


Starting training for 5 epochs...


Epoch 1/5 [Validating]: 100%|██████████| 550/550 [06:17<00:00,  1.46it/s, loss=0.394]  


Epoch 1/5 -> Train Loss: 0.4741, Train Accuracy: 0.8947 | Validation Loss: 0.1765, Validation Accuracy: 0.9499
-> New best model saved to 'best_crop_doctor_model.pth' with accuracy: 0.9499


Epoch 2/5 [Validating]: 100%|██████████| 550/550 [04:20<00:00,  2.11it/s, loss=0.286]  


Epoch 2/5 -> Train Loss: 0.2032, Train Accuracy: 0.9394 | Validation Loss: 0.1309, Validation Accuracy: 0.9606
-> New best model saved to 'best_crop_doctor_model.pth' with accuracy: 0.9606


...

Epoch 3/5 -> Train Loss: 0.1756, Train Accuracy: 0.9454 | Validation Loss: 0.1178, Validation Accuracy: 0.9627
-> New best model saved to 'best_crop_doctor_model.pth' with accuracy: 0.9627


Epoch 4/5 [Validating]: 100%|██████████| 550/550 [04:10<00:00,  2.19it/s, loss=0.171]   


Epoch 4/5 -> Train Loss: 0.1572, Train Accuracy: 0.9502 | Validation Loss: 0.1116, Validation Accuracy: 0.9636
-> New best model saved to 'best_crop_doctor_model.pth' with accuracy: 0.9636


Epoch 5/5 [Validating]: 100%|██████████| 550/550 [04:53<00:00,  1.87it/s, loss=0.138]   

Epoch 5/5 -> Train Loss: 0.1518, Train Accuracy: 0.9507 | Validation Loss: 0.0950, Validation Accuracy: 0.9685
-> New best model saved to 'best_crop_doctor_model.pth' with accuracy: 0.9685

Training complete!
Best validation accuracy: 0.9685


Testing on unseen data

In [21]:
# --- 1. Prepare the Test Data (Corrected with Filtering) ---
print("Loading test data and preparing the DataLoader...")

# Load the test DataFrame
test_df = pd.read_parquet(os.path.join(PROCESSED_DATA_DIR, 'test.parquet'))

# Get the list of valid, known class names from our mapping
valid_labels = class_to_idx.keys()

# **FILTERING STEP:** We only keep the rows in test_df that have a valid label.
original_size = len(test_df)
test_df_filtered = test_df[test_df['label'].isin(valid_labels)]
new_size = len(test_df_filtered)

if original_size != new_size:
    print(
        f"Filtered out {original_size - new_size} rows with invalid labels (like 'test').")

# Create the test dataset using the FILTERED dataframe
test_dataset = PlantVillageDataset(
    test_df_filtered, class_to_idx, transform=val_test_transforms)

# Create the test DataLoader
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

print(f"Test data ready. Found {len(test_dataset)} valid images.")

Loading test data and preparing the DataLoader...
Filtered out 33 rows with invalid labels (like 'test').
Test data ready. Found 0 valid images.


In [12]:
# --- 2. Load the Best Model ---

# Re-create the model architecture
# It must be the same architecture as the one we trained
# We don't need pre-trained weights now
model = models.efficientnet_b0(weights=None)
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(
    in_features=num_features, out_features=NUM_CLASSES)

# Define the path to the saved model
model_path = '../best_crop_doctor_model.pth'

# Load the saved state dictionary
model.load_state_dict(torch.load(model_path))

# Move the model to the correct device
model.to(device)

# Set the model to evaluation mode
# This is important as it disables layers like Dropout
model.eval()

print(
    f"Model loaded successfully from '{model_path}' and set to evaluation mode.")

Model loaded successfully from '../best_crop_doctor_model.pth' and set to evaluation mode.


In [13]:
# --- 3. Run Evaluation on the Test Set ---
test_preds = []
test_labels = []

print("\nMaking predictions on the test set...")

# We use torch.no_grad() to speed up inference and save memory
with torch.no_grad():
    # Wrap the test_loader with tqdm for a progress bar
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)

        # Get model outputs
        outputs = model(inputs)

        # Get the predicted class (the one with the highest score)
        _, preds = torch.max(outputs, 1)

        # Append predictions and true labels to our lists
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

print("Prediction on the test set complete.")


Making predictions on the test set...


Testing: 0it [00:00, ?it/s]

Prediction on the test set complete.


In [17]:
# --- FINAL EVALUATION ON THE VALIDATION SET ---
# Since the test set has label mismatches, we will use the validation
# set for our final performance report.

print("--- Evaluating model performance on the VALIDATION set ---")

# We will use the val_loader which we know is working correctly
val_preds = []
val_labels = []

model.eval()  # Make sure the model is in evaluation mode
with torch.no_grad():
    for inputs, labels in tqdm(val_loader, desc="Final Validation"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        val_preds.extend(preds.cpu().numpy())
        val_labels.extend(labels.cpu().numpy())

# --- Calculate and Display Final Metrics ---
# Calculate overall accuracy on the validation set
final_val_accuracy = accuracy_score(val_labels, val_preds)
print(
    f"\nFinal Model Accuracy (on validation set): {final_val_accuracy * 100:.2f}%")

# Generate the detailed classification report
idx_to_class = {i: label for label, i in class_to_idx.items()}
class_names = [idx_to_class[i].split('___')[1] for i in range(NUM_CLASSES)]

report = classification_report(
    val_labels,
    val_preds,
    target_names=class_names
)

print("\nDetailed Classification Report (on Validation Set):")
print(report)

--- Evaluating model performance on the VALIDATION set ---


Final Validation: 100%|██████████| 550/550 [05:32<00:00,  1.66it/s]


Final Model Accuracy (on validation set): 96.85%

Detailed Classification Report (on Validation Set):
                                      precision    recall  f1-score   support

                          Apple_scab       0.97      0.96      0.97       504
                           Black_rot       0.99      0.99      0.99       497
                    Cedar_apple_rust       0.98      0.97      0.98       440
                             healthy       0.96      0.98      0.97       502
                             healthy       0.98      0.98      0.98       454
                             healthy       0.98      1.00      0.99       456
                      Powdery_mildew       0.97      0.99      0.98       421
 Cercospora_leaf_spot Gray_leaf_spot       0.96      0.85      0.90       410
                        Common_rust_       0.99      0.99      0.99       477
                             healthy       0.99      1.00      1.00       465
                Northern_Leaf_Blight  

In [23]:
import re
from sklearn.metrics import classification_report, accuracy_score

# --- FINAL EVALUATION (Corrected Parsing Logic) ---

print("--- Preparing Test Data with Corrected File Name Parsing ---")


def create_test_df_from_filenames_corrected(directory, train_labels_map):
    """
    Parses a flat directory of test images where the label is in the filename.
    This version correctly handles the mapping logic.
    """
    filepaths = []
    labels = []

    # **THE FIX IS HERE:** This map now correctly includes the crop name.
    # e.g., 'Potato___Early_blight' -> 'potatoearlyblight'
    simple_name_map = {
        label.replace('___', '').replace('_', '').lower(): label
        for label in train_labels_map
    }

    for filename in os.listdir(directory):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            filepath = os.path.join(directory, filename)

            # This parsing logic is correct.
            # "PotatoEarlyBlight1.jpg" -> "potatoearlyblight"
            base_name = os.path.splitext(filename)[0]
            label_part = re.sub(
                r'\d+$', '', base_name).replace('_', '').lower()

            # Now, it will correctly find 'potatoearlyblight' in the map
            full_label = simple_name_map.get(label_part)

            if full_label:
                filepaths.append(filepath)
                labels.append(full_label)

    return pd.DataFrame({'filepath': filepaths, 'label': labels})


# 1. Define paths and get known labels
TEST_PATH = '../data/plantvillage_raw/test'
known_labels = class_to_idx.keys()

# 2. Create the test DataFrame using the CORRECTED custom function
test_df = create_test_df_from_filenames_corrected(TEST_PATH, known_labels)

print(f"Successfully parsed {len(test_df)} images from the test folder.")

# 3. Create the test_dataset and test_loader
test_dataset = PlantVillageDataset(
    test_df, class_to_idx, transform=val_test_transforms)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

# 4. Load the best model
model.load_state_dict(torch.load('../best_crop_doctor_model.pth'))
model.to(device)
model.eval()

# 5. Make predictions
test_preds = []
test_labels = []
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

# 6. Display the final report
test_accuracy = accuracy_score(test_labels, test_preds)
print(f"\n--- FINAL REPORT ---")
print(f"Overall Test Accuracy: {test_accuracy * 100:.2f}%")

class_names = [idx_to_class[i].split('___')[1] for i in range(NUM_CLASSES)]
report = classification_report(
    test_labels,
    test_preds,
    target_names=class_names,
    labels=range(NUM_CLASSES),
    zero_division=0
)

print("\nDetailed Classification Report (on Test Set):")
print(report)

--- Preparing Test Data with Corrected File Name Parsing ---
Successfully parsed 17 images from the test folder.


Testing: 100%|██████████| 1/1 [00:00<00:00,  1.70it/s]


--- FINAL REPORT ---
Overall Test Accuracy: 82.35%

Detailed Classification Report (on Test Set):
                                      precision    recall  f1-score   support

                          Apple_scab       0.00      0.00      0.00         0
                           Black_rot       0.00      0.00      0.00         0
                    Cedar_apple_rust       0.00      0.00      0.00         0
                             healthy       0.00      0.00      0.00         0
                             healthy       0.00      0.00      0.00         0
                             healthy       0.00      0.00      0.00         0
                      Powdery_mildew       0.00      0.00      0.00         0
 Cercospora_leaf_spot Gray_leaf_spot       0.00      0.00      0.00         0
                        Common_rust_       0.00      0.00      0.00         0
                             healthy       0.00      0.00      0.00         0
                Northern_Leaf_Blight      